In [1]:
s = "imad"

In [2]:
s[0:0], s[1:0]

('', '')

# Brute-Force

In [3]:
def find_brute_force(T, P):
    n, m = len(T), len(P)
    if m == 0:
        return 0
    for i in range(n - m + 1):
        j = 0
        while j < m and T[i + j] == P[j]:
            j += 1
        if j == m:
            return i
    return -1

The running time is $O(nm)$. However, for most applications it is proportional to $n$.

In [4]:
def find_brute_force_v1(T, P):
    n, m = len(T), len(P)
    if m == 0:
        return 0
    i = j = 0
    while i < n and j < m:
        if T[i] == P[j]:
            j += 1
        else:
            i -= j
            j = 0
        i += 1
    if j == m:
        return i - m
    return -1

In [5]:
T = "abacaabaccabacabaabb"
P = "abacab"

In [6]:
find_brute_force(T, P)

10

In [7]:
find_brute_force_v1(T, P)

10

# Boyer-Moore

In [8]:
def find_boyer_moore(T, P):
    n, m = len(T), len(P)
    if m == 0:
        return 0
    i = k = m - 1
    last = {k:i for i, k in enumerate(P)}
    while i < n:
        if T[i] == P[k]:
            if k == 0:
                return i
            else:
                i -= 1
                k -= 1
        else:
            j = last.get(T[i], -1)
            i += m - min(k, j + 1)
            k = m - 1
    return -1

In [9]:
find_boyer_moore(T, P)

10

It is still has $O(nm)$ in worst-case; however, for most application it is proportional $O(n + m)$.

# Knuth-Morris-Pratt

In [10]:
def compute_lps_table(P):
    """Computes the longest prefix suffix table for KMP algorithm."""
    m = len(P)
    lps_table = [0] * m
    k = 0
    j = 1
    while j < m:
        # match found
        if P[j] == P[k]:
            lps_table[j] = k + 1
            j += 1
            k += 1
        # k follows matching prefix
        elif k > 0:
            k = lps_table[k - 1]
        # no match found at index j
        else:
            j += 1
    return lps_table

In [11]:
def find_kmp(T, P):
    n, m = len(T), len(P)
    if m == 0:
        return 0
    lps_table = compute_lps_table(P)
    j = k = 0
    while j < n:
        if T[j] == P[k]:
            if k == m - 1:
                return j - m + 1
            j += 1
            k += 1
        elif k > 0:
            k = lps_table[k - 1]
        else:
            j += 1
    return -1

In [12]:
find_kmp(T, P)

10

The running time is $O(n + m)$ which is the best any algorithm can achieve since we still have to compare every character at least once in the worst-case.

# Rabin-Karp

In [13]:
def find_rabin_karp(T, P):
    """Return the index of first occurance of P; otherwise, returns -1."""
    n = len(T)
    m = len(P)
    t_hash = 0
    p_hash = 0
    h = 1
    d = 256  # Alphabet size
    q = 109_345_121  # Any large number >= m would work,
    # larger q --> lower probability for collisions

    for _ in range(m - 1):
        h = (h * d) % q

    # Calculate the hash value of the text and the pattern
    # for the first window:
    # p_hash = ord(P[0]) * d ** (m - 1) + ord(P[1]) * d ** (m - 2)
    for i in range(m):
        p_hash = (d * p_hash + ord(P[i])) % q
        t_hash = (d * t_hash + ord(T[i])) % q

    for i in range(n - m + 1):
        if p_hash == t_hash:
            # check if all characters are equal
            # (since hash values may be equal due to collisions)
            for k in range(m):
                if T[i + k] != P[k]:
                    break
            if k == m - 1:
                return i
        if i < n - m:
            # Rehashing: remove leading digit, add trailing digit
            t_hash = (d * (t_hash - ord(T[i]) * h) + ord(T[i + m])) % q
    return -1

In [14]:
find_rabin_karp(T, P)

10

The basic idea behind __Rabin-Karp__ algorithm for substring search is that we compare the hash values of m-substring with the pattern and every time we don't find a match we silde the window by 1 character to the right. However, computing the hash value for every m-substring of the text is slow. To make it run in constant time, we use rolling hashing: Every time we slide the window by one character, we delete the leading digit and add the training digit from the hash to compute the new hash. This would run in constant time. 

Below are the steps of the algorithms for a text of length $n$ and substring of length $m$:
1. Compute high-order digit that will be used when deleting the leading digit; $h = pow(d, m - 1) mod q$
2. We compute the hash values of the pattern (which will stay the same) and the first window of the text (m-substring). The alphabet size is typically 256 (ASCII) but can change depends on the application
3. We iterate over all $(n - m + 1)$ m-substrings to find the match
    1. If the hash values are equal, we chech the characters of window and the patter. This step is necessary since we may have collisions (spurious hit); which means different texts are mapped to the same value
        1. If the characters are equal, then we find a match and return
    2. else, recompute the hash value of the new window using rolling hashing.

To reduce the probability of collisions, we pick a very large prime number 

Rabin-Karp (Las Vegas) running time is $O(n + m)$ for best case and average case; however, it is $O(nm)$ in the worst case where the hash function returns the same hash value for each window and we need to do the comparison for each window to check whether the pattern matches. To reduce collision and reduce the spurious hits, we choose __q__ to be large prime number.

Even though Rabin-Karp algorithm has the same (expected) running time as KMP, KMP is faster because of bigger constant factor due to the more operations done in the inner loop.

# Summary

The table at the bottom of the page summarizes the algorithms that we have discussed for substring search. As is often the case when we have several algorithms for the same task, each of them has attractive features. 
1. Brute-force search is easy to implement and works well in typical cases (Java’s indexOf() method in String uses brute-force search); however, it might require time proportional to MN
2. Knuth-Morris-Pratt is guaranteed linear-time with no backup in the input
3. Boyer-Moore is sublinear (by a factor of M) in typical situations
4. Rabin- Karp is linear

Knuth-Morris-Pratt and Boyer-Moore use extra space; and Rabin-Karp has a relatively long inner loop (several arithmetic operations, as opposed to character compares in the other methods).
<img src="substrinng_search_running_time_summary.png">